# Overview

This notebook is based on this [Prediction and Feature Store Online Serving](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/mobile_gaming/mobile_gaming_feature_store.ipynb) notebook and [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)

### Dataset

The dataset is the public sample export data from an actual mobile game app called "Flood It!" (Android, iOS)

### Model

The model you use in this notebook is based on [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml). The idea behind this model is that your company has extensive log data describing how your game users have interacted with the site. The raw data contains the following categories of information:

- identity - unique player identitity numbers
- demographic features - information about the player, such as the geographic region in which a player is located
- behavioral features - counts of the number of times a  player has triggered certain game events, such as reaching a new level
- churn propensity - this is the label or target feature, it provides an estimated probability that this player will churn, i.e. stop being an active player.

The blog article referenced above explains how to use BigQuery to store the raw data, pre-process the data for machine learning, and train the corresponding model. Because this notebook focuses on model monitoring, rather than training models, you're going to reuse a pre-trained version of this model, which has been exported to Cloud Storage, which is stored in `gs://mco-mm/churn`

# Basic set up

## Install packages

In [ ]:
! pip install google-cloud-aiplatform

## Set up project

In [ ]:
import os

PROJECT_ID = "bon-test-0"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

In [ ]:
! gcloud config set project $PROJECT_ID

The feature is only available in `us-central1` now

In [ ]:
REGION = "us-central1"

## Timestamp

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

## Authenticate your Google Cloud account

If you are using Vertex AI Workbench Notebooks, your environment is already authenticated. Skip this step.

If you are using Colab, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

Otherwise, follow these steps:

1. In the Cloud Console, go to the Create service account key page.

2. Click Create service account.

3. In the Service account name field, enter a name, and click Create.

4. In the Grant this service account access to project section, click the Role drop-down list and add the following roles:

- BigQuery Admin
- Storage Admin
- Storage Object Admin
- Vertex AI Administrator
- Vertex AI Feature Store Admin

5. Click Create. A JSON file that contains your key downloads to your local environment.

6. Enter the path to your service account key as the `GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

Create a bucket and copy the exported model to it

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

Only if your bucket doesn't already exist: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

In [ ]:
! gsutil ls -al $BUCKET_URI

Copy the trained model to your bucket

In [ ]:
! gsutil cp -r gs://mco-mm/churn/* $BUCKET_URI

### Create a service account

We need a service account for this new feature because the prediction workload's credential does not have access to Feature Store. Create one and grant the role `roles/aiplatform.serviceAgent` to it, which will grant it the access to most prediction's resources and Feature Store.

In [ ]:
SA_NAME = "prediction-and-fs-testing"
SA_DESCRIPTION = '"SA to test Prediction and Feature Store integration"'
DISPLAY_NAME = "prediction-and-fs-testing"

In [ ]:
!gcloud iam service-accounts create $SA_NAME \
    --description=$SA_DESCRIPTION \
    --display-name=$DISPLAY_NAME

In [ ]:
# SERVICE_ACCOUNT = "prediction-and-fs@bon-test-0.iam.gserviceaccount.com"
SERVICE_ACCOUNT = (
    f"{SA_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"  # @param {type:"string"}
)

Grant the Service account the `Storage Admin` and `Vertex AI Feature Store Data Viewer` role to access the artifacts in GCS and data in Feature Store

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/storage.objectAdmin;

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/aiplatform.featurestoreDataViewer;

## Import libraries

In [ ]:
# General
import os
import sys

# Vertex AI and its Feature Store
from google.cloud import aiplatform
from google.cloud.aiplatform import Featurestore

**NOTE:** The feature is only available in `autopush` now

In [ ]:
aiplatform.constants.base.API_BASE_PATH = "autopush-aiplatform.sandbox.googleapis.com"
aiplatform.constants.base.PREDICTION_API_BASE_PATH = (
    "autopush-prediction-aiplatform.sandbox.googleapis.com"
)

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

# Create Feature Store

In [ ]:
FEATURESTORE_ID = "mobile_gaming_" + TIMESTAMP  # @param {type:"string"}

# Vertex AI Feature store
ONLINE_STORE_NODES_COUNT = 5
DEMOGRAPHIC_ENTITY_ID = "demographic"
BEHAVIOR_ENTITY_ID = "behavior"
FEATURE_TIME = "timestamp"
ENTITY_ID_FIELD = "user_pseudo_id"

We will use the exported sample data of the dataset used in this [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)

In [ ]:
SOURCE_URI = "gs://featurestore_prediction_integration/data/mobile_gaming_dataset.csv"

## Create Feature Store

In [ ]:
try:
    mobile_gaming_feature_store = Featurestore.create(
        featurestore_id=FEATURESTORE_ID,
        online_store_fixed_node_count=ONLINE_STORE_NODES_COUNT,
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    FEATURESTORE_RESOURCE_NAME = mobile_gaming_feature_store.resource_name
    print(f"Feature store created: {FEATURESTORE_RESOURCE_NAME}")

### Create Entities

In [ ]:
try:
    demographic_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=DEMOGRAPHIC_ENTITY_ID,
        description="User demographic Entity",
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    DEMOGRAPHIC_ENTITY_RESOURCE_NAME = demographic_entity_type.resource_name
    print("Entity type name is", DEMOGRAPHIC_ENTITY_RESOURCE_NAME)

In [ ]:
try:
    behavior_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=BEHAVIOR_ENTITY_ID, description="User behavior Entity", sync=True
    )
except RuntimeError as error:
    print(error)
else:
    BEHAVIOR_ENTITY_RESOURCE_NAME = behavior_entity_type.resource_name
    print("Entity type name is", BEHAVIOR_ENTITY_RESOURCE_NAME)

### Create Features

#### Feature Config

In [ ]:
demographic_feature_configs = {
    "country": {
        "value_type": "STRING",
        "description": "The country of customer",
        "labels": {"status": "passed"},
    },
    "operating_system": {
        "value_type": "STRING",
        "description": "The operating system of device",
        "labels": {"status": "passed"},
    },
    "language": {
        "value_type": "STRING",
        "description": "The language of device",
        "labels": {"status": "passed"},
    },
    "user_pseudo_id": {
        "value_type": "STRING",
        "description": "User pseudo id",
        "labels": {"status": "passed"},
    },
}

behavior_feature_configs = {
    "cnt_user_engagement": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement level",
        "labels": {"status": "passed"},
    },
    "cnt_level_start_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with start level",
        "labels": {"status": "passed"},
    },
    "cnt_level_end_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with end level",
        "labels": {"status": "passed"},
    },
    "cnt_level_complete_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with complete status",
        "labels": {"status": "passed"},
    },
    "cnt_level_reset_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with reset status",
        "labels": {"status": "passed"},
    },
    "cnt_post_score": {
        "value_type": "DOUBLE",
        "description": "A variable of user score",
        "labels": {"status": "passed"},
    },
    "cnt_spend_virtual_currency": {
        "value_type": "DOUBLE",
        "description": "A variable of user virtual amount",
        "labels": {"status": "passed"},
    },
    "cnt_ad_reward": {
        "value_type": "DOUBLE",
        "description": "A variable of user reward",
        "labels": {"status": "passed"},
    },
    "cnt_challenge_a_friend": {
        "value_type": "DOUBLE",
        "description": "A variable of user challenges with friends",
        "labels": {"status": "passed"},
    },
    "cnt_completed_5_levels": {
        "value_type": "DOUBLE",
        "description": "A variable of user level 5 completed",
        "labels": {"status": "passed"},
    },
    "cnt_use_extra_steps": {
        "value_type": "DOUBLE",
        "description": "A variable of user extra steps",
        "labels": {"status": "passed"},
    },
    "month": {
        "value_type": "INT64",
        "description": "First touch month",
        "labels": {"status": "passed"},
    },
    "julianday": {
        "value_type": "INT64",
        "description": "First touch julian day",
        "labels": {"status": "passed"},
    },
    "dayofweek": {
        "value_type": "INT64",
        "description": "First touch day of week",
        "labels": {"status": "passed"},
    },
}

#### Create features using `batch_create_features` method

In [ ]:
try:
    demographic_entity_type.batch_create_features(
        feature_configs=demographic_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in demographic_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

In [ ]:
try:
    behavior_entity_type.batch_create_features(
        feature_configs=behavior_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in behavior_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

### Ingest features 

In [ ]:
DEMOGRAPHIC_FEATURES_IDS = [
    feature.name for feature in demographic_entity_type.list_features()
]

In [ ]:
try:
    demographic_entity_type.ingest_from_gcs(
        feature_ids=DEMOGRAPHIC_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
    # demographic_entity_type.ingest_from_bq(
    #     feature_ids=DEMOGRAPHIC_FEATURES_IDS,
    #     feature_time=FEATURE_TIME,
    #     bq_source_uri=BQ_SOURCE_URI,
    #     entity_id_field=ENTITY_ID_FIELD,
    #     disable_online_serving=False,
    #     worker_count=10,
    #     sync=False,
    # )
except RuntimeError as error:
    print(error)

In [ ]:
BEHAVIOR_FEATURES_IDS = [
    feature.name for feature in behavior_entity_type.list_features()
]

In [ ]:
try:
    behavior_entity_type.ingest_from_gcs(
        feature_ids=BEHAVIOR_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
    # behavior_entity_type.ingest_from_bq(
    #     feature_ids=BEHAVIOR_FEATURES_IDS,
    #     feature_time=FEATURE_TIME,
    #     bq_source_uri=BQ_SOURCE_URI,
    #     entity_id_field=ENTITY_ID_FIELD,
    #     disable_online_serving=False,
    #     worker_count=10,
    #     sync=False,
    # )
except RuntimeError as error:
    print(error)

# Create Feature Fetch Config

## Feature fetch config proto:

```protobuf
message FeatureFetchConfig {
  // The format of the internal prediction request auto-created after features
  // are fetched. Prediction currently supports XGBoost, TensorFlow and
  // scikit-learn, and will soon start to support Pytorch. Among these
  // frameworks, XGBoost supports array input format only (i.e. input
  // features are in the form of an array), whereas the other three frameworks
  // can allow both dictionary format and array format inputs.
  ModelInputFormat model_input_format = 3;
  enum ModelInputFormat {
    MODEL_INPUT_FORMAT_UNSPECIFIED = 0;
    ARRAY = 1;
    DICT = 2;
  }

  // Specifying details of the prediction input
  repeated Feature features = 4;
  message Feature {
    // When internal_request_format = DICT, this value_key is used
    // for the Internal Prediction Request as the key to the feature value.
    // In the FeatureFetchConfig, a pass-through feature can be represented by
    // a Feature message with just a value_key.
    string value_key = 1;

    // Defines where from featurestore(s) does each feature comes.
    FeatureSource feature_source = 2;
    message FeatureSource {
      // From a high level, there are two fields in FeatureSource, where
      // entity_id_key is about the "row" from where a value is fetched, and
      // feature_resource_path is about the "column" from where a value is
      // fetched.

      // Specifies which key holds the entity ID in the external request sent by
      // the user to the prediction service.
      string entity_id_key = 1;

      // The resource path in URL format, to identify the entity type.
      // The format should be
      // projects/PROJECT/locations/LOCATION/featurestores/FEATURESTORE_ID/entityTypes/ENTITY_TYPE_ID/
      // e.g.
      // "projects/my-feature-store-project/locations/us-central1/featurestores/movie_predictions/entityTypes/movies/"
      string entity_type = 2;

      // The feature ID defined in feature store.
      string feature_id = 3;
    }
  }
}
```

## Generate Feature fetch config

In [ ]:
FEATURE_FETCH_CONFIG_TEMPLATE = """modelInputFormat: DICT
features:
- valueKey: country
  featureSource:
    entityIdKey: demographic
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    featureId: country
- valueKey: operating_system
  featureSource:
    entityIdKey: demographic
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    featureId: operating_system
- valueKey: language
  featureSource:
    entityIdKey: demographic
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    featureId: language
- valueKey: user_pseudo_id
  featureSource:
    entityIdKey: demographic
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    featureId: user_pseudo_id
- valueKey: cnt_user_engagement
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_user_engagement
- valueKey: cnt_level_start_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_start_quickplay
- valueKey: cnt_level_end_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_end_quickplay
- valueKey: cnt_level_complete_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_complete_quickplay
- valueKey: cnt_level_reset_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_reset_quickplay
- valueKey: cnt_post_score
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_post_score
- valueKey: cnt_spend_virtual_currency
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_spend_virtual_currency
- valueKey: cnt_ad_reward
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_ad_reward
- valueKey: cnt_challenge_a_friend
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_challenge_a_friend
- valueKey: cnt_completed_5_levels
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_completed_5_levels
- valueKey: cnt_use_extra_steps
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_use_extra_steps
- valueKey: month
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: month
- valueKey: julianday
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: julianday
- valueKey: dayofweek
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: dayofweek"""

In [ ]:
feature_fetch_config = FEATURE_FETCH_CONFIG_TEMPLATE.format(
    PROJECT_NUMBER=PROJECT_NUMBER, REGION=REGION, FEATURESTORE_ID=FEATURESTORE_ID
)

In [ ]:
print(feature_fetch_config)

In [ ]:
with open("prediction_featuresstore_fetch_config.yaml", "w") as f:
    f.write(feature_fetch_config)

In [ ]:
# Remove if the file already exists
!gsutil rm $BUCKET_URI/prediction_featuresstore_fetch_config.yaml

In [ ]:
!gsutil cp prediction_featuresstore_fetch_config.yaml $BUCKET_URI

# Integrate with Vertex Prediction

## Upload Model

In [ ]:
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"
DISPLAY_NAME = "mobile_gaming_featureStore_integration_"

In [ ]:
model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME + TIMESTAMP,
    artifact_uri=BUCKET_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

## Online Prediction

### Deploy Model

In [ ]:
DEPLOYED_NAME = DISPLAY_NAME + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MACHINE_TYPE = "n1-standard-4"

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    service_account=SERVICE_ACCOUNT,
)

### Predict

In [ ]:
DEFAULT_PRED_REQUEST = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "country": "Australia",
        "operating_system": "IOS",
        "language": "en-au",
        "cnt_user_engagement": 3.0,
        "cnt_level_start_quickplay": 1.0,
        "cnt_level_end_quickplay": 0.0,
        "cnt_level_complete_quickplay": 0.0,
        "cnt_level_reset_quickplay": 0.0,
        "cnt_post_score": 0.0,
        "cnt_spend_virtual_currency": 0.0,
        "cnt_ad_reward": 0.0,
        "cnt_challenge_a_friend": 0.0,
        "cnt_completed_5_levels": 0.0,
        "cnt_use_extra_steps": 0.0,
        "month": 7,
        "julianday": 194,
        "dayofweek": 6,
    },
    {
        "user_pseudo_id": "E5D16173400729E05DFFB4883FA6EF1C",
        "country": "United States",
        "operating_system": "IOS",
        "language": "en-us",
        "cnt_user_engagement": 1.0,
        "cnt_level_start_quickplay": 1.0,
        "cnt_level_end_quickplay": 0.0,
        "cnt_level_complete_quickplay": 0.0,
        "cnt_level_reset_quickplay": 0.0,
        "cnt_post_score": 0.0,
        "cnt_spend_virtual_currency": 0.0,
        "cnt_ad_reward": 0.0,
        "cnt_challenge_a_friend": 0.0,
        "cnt_completed_5_levels": 0.0,
        "cnt_use_extra_steps": 0.0,
        "month": 6,
        "julianday": 173,
        "dayofweek": 6,
    },
]

In [ ]:
FS_PRED_REQUEST = [
    {
        "demographic": "AB0F2EE5F9F401763BE1E9FA55410312",
        "behavior": "AB0F2EE5F9F401763BE1E9FA55410312",
    },
    {
        "demographic": "E5D16173400729E05DFFB4883FA6EF1C",
        "behavior": "E5D16173400729E05DFFB4883FA6EF1C",
    },
]

In [ ]:
endpoint.predict([DEFAULT_PRED_REQUEST[0]])

In [ ]:
DEFAULT_RESPONSE = endpoint.predict(DEFAULT_PRED_REQUEST)

In [ ]:
DEFAULT_RESPONSE

### Single instance

In [ ]:
endpoint.predict([FS_PRED_REQUEST[0]])

### Multiple instances

In [ ]:
FS_RESPONSE = endpoint.predict(FS_PRED_REQUEST)

In [ ]:
FS_RESPONSE

### Compare response

In [ ]:
DEFAULT_RESPONSE.predictions == FS_RESPONSE.predictions

# Clean Up

In [ ]:
# delete feature store
mobile_gaming_feature_store.delete(sync=True, force=True)

In [ ]:
# delete Vertex AI resources
endpoint.undeploy_all()
endpoint.delete()
model.delete

In [ ]:
# Delete bucket
!gsutil -m rm -r $BUCKET_URI